In [ ]:
import os
import re

In [ ]:
import spacy
from spacy.matcher import Matcher
#import en_core_web_lg
import en_core_web_sm


2.1.8


In [ ]:
!python -m spacy download en_core_web_sm


     |████████████████████████████████| 11.1MB 3.8MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-cp36-none-any.whl size=11074435 sha256=2da1df0062bee99ecb0900ec96b8b2f5baeb6ac8eb33a9de372325bcc997935a
  Stored in directory: /tmp/pip-ephem-wheel-cache-fqb4z8iv/wheels/39/ea/3b/507f7df78be8631a7a3d7090962194cf55bc1158572c0be77f
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
!pip install tika


In [ ]:
from tika import parser 

raw = parser.from_file('./Data/1404800252689.pdf')
text = raw['content']

In [ ]:
#All AO orders start with a background subsection 
header_index = text.lower().find('background')
#print(header_index)
text_header = text[:header_index]
text = text[header_index:]
points_split = re.split("\n[0-9]?[0-9]\.\s",text)
num_of_points = len(points_split)
#print(text_header)

In [ ]:



def preprocess(pt):
  pt = re.sub('\[.*\]','',pt)
  pt = re.sub('\(.*\)','',pt)
  pt = re.sub('\n','',pt)
  pt = re.sub('Order\%.*\%.',"",pt)
  pt = re.sub('Microsoft Word',"",pt)  
  pt = re.sub('\sPage\s[0-9]\sof\s[0-9]?[0-9]\s','',pt)
  pt = re.sub('.*docx','',pt)
  pt = re.sub('_*_','',pt)
  pt = re.sub('/',' ',pt)
  pt = re.sub('\sPage\s[0-9]','',pt)
  pt = re.sub('[0-9]?[0-9]\.','',pt)
  pt = pt.lstrip()
  pt = pt.rstrip()
  return pt

pre_processed_header = preprocess(text_header)

In [ ]:
print(pre_processed_header)

AO order  Orchid Chemicals and Pharmaceuticals Senthil Kumar  BEFORE THE ADJUDICATING OFFICER SECURITIES AND EXCHANGE BOARD OF INDIA   [ADJUDICATION ORDER NO. PGUNDER SECTION 15-I OF SECURITIES AND EXCHANGE BOARD OF INDIA ACT, 1992 READ WITH RULE 5 OF SECURITIES AND EXCHANGE BOARD OF INDIA (PROCEDURE FOR HOLDING INQUIRY AND IMPOSING PENALTIES BY ADJUDICATING  RULES, 1995  AO SPV 71 2012]  In respect of  Mr. Senthil Kumar U.P.                                In the matter of            M s Orchid Chemicals and Pharmaceuticals Limited


In [ ]:
nlp = en_core_web_sm.load()
matcher = Matcher(nlp.vocab)
doc = nlp(pre_processed_header)
#print(doc)

In [ ]:
pattern_names = [{"LOWER": "in"}, {"LEMMA": "respect"},{"LEMMA": "of"}]
matcher.add("Names", None, pattern_names)
matches = matcher(doc)
for match_id, start, end in matches:
  names_start = end 

pattern_company = [{"LOWER": "in"}, {"LEMMA": "the"},{"LEMMA": "matter"}, {"LEMMA": "of"}]
matcher.add("Names", None, pattern_company)
matches = matcher(doc)
for match_id, start, end in matches:
  names_end = start
  company_starts = end

tokens = [token.text for token in doc] 
print(f'The names of the Noticees is/are {" ".join(tokens[names_start:names_end])}')
print(f'The name of the company is {" ".join(tokens[company_starts:])}')




The names of the Noticees is/are   Mr. Senthil Kumar U.P.                                
The name of the company is             M s Orchid Chemicals and Pharmaceuticals Limited


In [ ]:
#validating spacys NER 
names_spacy = [ent for ent in doc.ents if ent.label_ == "PERSON"]
print(names_spacy)
Organization_spacy = [ent for ent in doc.ents if ent.label_ == "ORG"]
print(Organization_spacy)

[Kumar  , READ, Senthil Kumar]
[AO, SECURITIES AND, Orchid Chemicals and Pharmaceuticals Limited]


In [ ]:
def footer_preprocess(pt):
  pt = re.sub('\n','',pt)
  pt = re.sub('Order\%.*\%.',"",pt)
  pt = re.sub('\sPage\s[0-9]\sof\s[0-9]?[0-9]\s',"",pt)
  pt = pt.lstrip()
  pt = pt.rstrip()
  pt = " ".join(pt.split())
  return pt

footer_pre_processed = footer_preprocess(points_split[num_of_points-1])
print(footer_pre_processed)

In terms of the Rule 6 of the Adjudication Rules, copies of this order are sent to the Noticee and also to the Securities and Exchange Board of India. The matter is disposed of accordingly. DATE: November 29, 2012 PIYOOSH GUPTA PLACE: Mumbai ADJUDICATING OFFICER BEFORE THE ADJUDICATING OFFICER


In [ ]:
doc = nlp(footer_pre_processed)
pattern_place = [{"LOWER": "place"}, {'IS_PUNCT': True}]
matcher.add("place", None, pattern_place)
matches = matcher(doc)
for match_id, start, end in matches:
  place_index = end
tokens = [token.text for token in doc]
print(f"The place is {tokens[place_index]}")

#Only complete rule 
pattern_date = [{'LOWER': 'date'},
          {'IS_PUNCT': True},
          {'ENT_TYPE': 'DATE'},
          {'ENT_TYPE': 'DATE'},
          {'IS_PUNCT': True},
          {'IS_DIGIT': True}]
matcher.add("date",None,pattern_date)
matches = matcher(doc)
for match_id, start, end in matches:
  if tokens[start+2:end]:
    print(f"The date is {' '.join(tokens[start+2:end])}")

The place is Mumbai
The date is November 29 , 2012


In [ ]:
#validating spacys NER 

locations_spacy = [ent for ent in doc.ents if ent.label_ == "LOC"]
print(locations_spacy)
Gpe_spacy = [ent for ent in doc.ents if ent.label_ == "GPE"]
print(Gpe_spacy)
dates_spacy = [ent for ent in doc.ents if ent.label_ == "DATE"]
print(dates_spacy)


[]
[India, Mumbai]
[November 29, 2012]


In [ ]:
results = re.findall('\n\n[Aa1iI]\.',points_split[4].lower())
print(results)

['\n\na.']


In [ ]:
!pip3 install https://blackstone-model.s3-eu-west-1.amazonaws.com/en_blackstone_proto-0.0.1.tar.gz



  Using cached https://blackstone-model.s3-eu-west-1.amazonaws.com/en_blackstone_proto-0.0.1.tar.gz
  Created wheel for en-blackstone-proto: filename=en_blackstone_proto-0.0.1-cp36-none-any.whl size=243759404 sha256=d99a0b46a5fffc1c06c1a570c24bdbd7523dac586c1467926aa0d3929a5eaa19
  Stored in directory: /root/.cache/pip/wheels/a2/81/dd/09c3b4ef7899e7d9cf92ed3152d29a08b5fd80f7a8bf66df4d
Successfully built en-blackstone-proto


In [ ]:
import en_blackstone_proto

In [ ]:

matcher = Matcher(nlp.vocab)
for pt in points_split[1:num_of_points-1]:
  doc = nlp(pt)
  tokens = [token.text for token in doc]
  pattern_regulations_one = [{'LOWER': 'a.'},
            {'LOWER': 'whether'}]
  pattern_regulations_two = [{'LOWER': 'a)'},
            {'LOWER': 'whether'}]
  pattern_regulations_three = [{'LOWER': '1.'},
            {'LOWER': 'whether'}]
  pattern_regulations_four = [{'LOWER': '1)'},
            {'LOWER': 'whether'}]

  matcher.add("RegRule1",None,pattern_regulations_one)
  matcher.add("RegRule2",None,pattern_regulations_two)
  matcher.add("RegRule3",None,pattern_regulations_three)
  matcher.add("RegRule4",None,pattern_regulations_four)
  matches = matcher(doc)
  if len(matches)>0:
    print("found")

#load blackstone and run where found get a list of regulations 
#Use file to write 
#notice the irregularities 

    


found


In [ ]:
all_files = []
is_found = False

for path,dirname,filenames in os.walk('/content/drive/My Drive/SEBI /Orders'):
  for names in filenames:
    all_files.append(os.path.join(path,names))

all_regulation_text = []
for document in all_files:
  is_found == False
  raw = parser.from_file(document)
  text = raw['content']with
  #All AO orders start with a background subsection 
  header_index = text.lower().find('background')
  if header_index == -1:
    header_index = text.lower().find('facts')
  #print(header_index)
  text_header = text[:header_index]
  text = text[header_index:]
  points_split = re.split("\n[0-9]?[0-9]\.\s",text)
  num_of_points = len(points_split)
  #print(text_header)
  nlp = en_core_web_sm.load()
  matcher = Matcher(nlp.vocab)
  regulation_text_point = []
  for pt in points_split[1:num_of_points-1]:
    is_found = False 
    pt = re.sub('\n',' ',pt)
    doc = nlp(pt)
    tokens = [token.text for token in doc]
    pattern_regulations_one = [{'LOWER': 'a.'},
              {'LOWER': 'whether'}]
    pattern_regulations_two = [{'ORTH': 'a'},
           {'IS_PUNCT': True},
           {'IS_PUNCT': True, 'OP': '?'},
           {'LOWER': 'whether'}]
    pattern_regulations_three = [{'LOWER': '1.'},
              {'LOWER': 'whether'}]
    pattern_regulations_four = [{'ORTH': '1'},
           {'IS_PUNCT': True},
           {'LOWER': 'whether'}]
    pattern_regulations_five = [{'LOWER': '1)'},
              {'LOWER': 'whether'}]
    pattern_regulations_six = [{'ORTH': 'i'},
           {'IS_PUNCT': True},
           {'IS_PUNCT': True, 'OP': '?'},
           {'LOWER': 'whether'}]
    pattern_regulations_seven = [{'IS_PUNCT': False, 'IS_ALPHA': False, 'IS_DIGIT': False, 'IS_ASCII': False},
           {'LOWER': 'whether'}]
    matcher.add("RegRule1",None,pattern_regulations_one)
    matcher.add("RegRule2",None,pattern_regulations_two)
    matcher.add("RegRule3",None,pattern_regulations_three)
    matcher.add("RegRule4",None,pattern_regulations_four)
    matcher.add("RegRule5",None,pattern_regulations_five)
    matcher.add("RegRule6",None,pattern_regulations_six)
    matcher.add("RegRule7",None,pattern_regulations_seven)

    matches = matcher(doc)
    if len(matches)>0:
      print("found")
      regulation_text_point.append(pt)
  all_regulation_text.append((regulation_text_point,document))

  
  

   

found
found
found
found
found
found
found
found
found
found
found
found
found
found
found
found


In [ ]:
nlp = en_blackstone_proto.load()
for regtext in all_regulation_text:

  total_ents = []
  print(regtext[0])
  for text in regtext[0]:
    doc = nlp(text)
    for ent in doc.ents:
      if ent.label_ == 'PROVISION':
        total_ents.append(ent.text)

  print(total_ents)
  print(regtext[1],end="\n")


['On perusal of the Show Cause Notice, and other material available on record, I have the  following issues for consideration, viz,      (1) Whether the noticee has violated Regulations 3 (a) (b) (c) & (d), 4 (1), (2) (a) (d) (k) and   (r) of SEBI (PFUTP) Regulations 2003.  (2) Whether the noticee has violated Section 11C(3) read with 11C(2) of the SEBI Act.  (3) Whether the noticee has violated Regulation 13(4) read with 13(5) of SEBI (PIT)   Regulations, 1992  (4) Whether the noticee is liable for monetary penalty under sections. 15HA and 15A (a) &   (b) of SEBI Act, 1992. .  (5) What quantum of monetary penalty should be imposed on the noticee, taking into   consideration the factors mentioned in section 15J of SEBI Act     FINDINGS  ']
['Regulations 3', 'Section 11C(3)', 'Regulation 13(4)', 'section 15J']
/content/drive/My Drive/SEBI /Orders/1404287891786.pdf

['I have carefully perused the documents available on record and   the submissions of the Noticee. The issues that arise fo